In [1]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

2023-12-31 18:33:21.676516: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-31 18:33:22.222012: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Change fonts and specify font size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
FONT_SIZE = 12

In [3]:
# Define function to normalize the input and output dataset
def normalize_data(data_used_to_normalize, data_to_be_normalized, type_of_data_subset):

    data_mean = np.mean(data_used_to_normalize)
    data_range = np.max(data_used_to_normalize) - np.min(data_used_to_normalize)

    normalized_data = (data_to_be_normalized - data_mean) / data_range
    normalizing_parameters = np.array(list([data_mean, data_range]))

    print(f'Normalized {type_of_data_subset} dataset.')

    return normalized_data, normalizing_parameters

In [4]:
# Define directories
current_directory = os.getcwd()

# Define directory for the assembled data
assembled_data_directory = os.path.join(current_directory, '..', '..', 'data', 'assembled')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'training_results')

In [5]:
# Load the training subsets for both displacement and force
training_displacement_data = np.load(os.path.join(assembled_data_directory, 'training_displacement_data.npy'))
training_force_data = np.load(os.path.join(assembled_data_directory, 'training_force_data.npy'))

In [7]:
# Print the shapes of the displacement and force data
print(f'The shape of displacement data is {training_displacement_data.shape[1:]}.')
print(f'The shape of force data is {training_force_data.shape[1:]}.')

The shape of displacement data is (501, 74).
The shape of force data is (501, 894).
